# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from citipy import citipy

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output_data/weather_data.csv')
weather_df.head()

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Max Temp (F),Latitude,Longitude,Wind Speed (mph)
0,0,luderitz,0,NaN,1584036397,64,71.60,-26.65,15.16,18.34
1,1,victoria,40,HK,1584036351,94,71.60,22.29,114.16,13.87
2,2,carnarvon,100,AU,1584036143,69,80.60,-24.87,113.63,14.99
3,3,semme,75,FR,1584036398,93,48.99,46.20,1.50,9.17
4,4,rengali,33,IN,1584036398,67,75.20,21.63,84.05,4.25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key= g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Latitude", "Longitude"]]

# HINT: be sure to handle NaN values
#census_data_complete = census_data_complete.dropna()
#poverty_rate = census_data_complete["Poverty Rate"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity (%)"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_df = weather_df.dropna()

filter_weather_df = weather_df[(weather_df["Max Temp (F)"] > 70) & 
                        (weather_df["Max Temp (F)"] < 80) &  
                        (weather_df["Wind Speed (mph)"] < 10) &
                        (weather_df["Cloudiness (%)"] == 0)
                       ]
filter_weather_df

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Max Temp (F),Latitude,Longitude,Wind Speed (mph)
78,78,muscat,0,OM,1584036408,46,73.40,23.61,58.59,2.24
99,99,antri,0,IN,1584036411,22,71.78,26.06,78.21,9.82
166,166,dicabisagan,0,PH,1584036125,83,75.56,17.08,122.42,8.63
230,230,dawei,0,MM,1584036428,73,78.62,14.08,98.20,4.88
358,358,yulara,0,AU,1584036349,25,75.20,-25.24,130.99,5.82
419,419,matara,0,LK,1584036451,81,75.56,5.95,80.54,7.96
459,459,kununurra,0,AU,1584036456,88,75.20,-15.77,128.73,1.97
467,467,mount isa,0,AU,1584036457,88,71.60,-20.73,139.50,8.19
477,477,betioky,0,MG,1584036458,50,78.80,-23.72,44.38,3.80
491,491,aloleng,0,PH,1584036460,72,73.22,16.13,119.78,3.09


In [7]:
filter_weather_df['Hotel'] = ""
filter_weather_df.head()

C:\Users\lujan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Max Temp (F),Latitude,Longitude,Wind Speed (mph),Hotel
78,78,muscat,0,OM,1584036408,46,73.40,23.61,58.59,2.24,
99,99,antri,0,IN,1584036411,22,71.78,26.06,78.21,9.82,
166,166,dicabisagan,0,PH,1584036125,83,75.56,17.08,122.42,8.63,
230,230,dawei,0,MM,1584036428,73,78.62,14.08,98.20,4.88,
358,358,yulara,0,AU,1584036349,25,75.20,-25.24,130.99,5.82,


In [57]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "23.61,58.59",
    "rankby": "distance",
    "type": "hotel",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map